In [1]:
def get_retention_rate(start_time: str, 
                       finish_time=datetime.now().strftime('%Y-%m-%d'), 
                       limit_days=None,
                       reg_file_path='~/shared/problem1-reg_data.csv', 
                       auth_file_path='~/shared/problem1-auth_data.csv'):
    '''Функция принимает на вход начальную дату отслеживания retantion - start_time, 
    конечную дату отслеживания retantion - finish_time,
    необходимое количество исследуемых дней с момента регистрации пользователем limit_days,
    путь к файлу с регистрациями пользователей - reg_file_path, 
    путь к файлу с авторизациями пользователей - auth_file_path.
    Функция возвращает тепловую карту с retantion.
    '''
    #  Считываем оба датафрейма
    df_reg = pd.read_csv(reg_file_path, sep=';')
    df_auth = pd.read_csv(auth_file_path, sep=';')
    
    #  На всякий случай удаляем из датафреймов пропущенные строки
    df_reg.dropna(inplace=True)
    df_auth.dropna(inplace=True)
   
    #  Объединяем два датафрейма в один
    full_users_data = df_auth.merge(df_reg, on='uid', how='left').sort_values(['uid', 'auth_ts'])
    
    #  Добавим колонки с датами регистрации и авторизации пользователем
    lambda_func = lambda x: dt.date.fromtimestamp(x)
    full_users_data['reg_date'] = pd.to_datetime(full_users_data['reg_ts'].apply(lambda_func))
    full_users_data['auth_date'] = pd.to_datetime(full_users_data['auth_ts'].apply(lambda_func))
    
    #  Оставим в датафрейме только необходимые данные
    full_users_data = full_users_data.query('reg_date >= @start_time & auth_date <= @finish_time')
        
    #  Добавим колонку с количеством дней от регистрации до авторизации
    full_users_data['DBRA'] = (full_users_data['auth_date'] - full_users_data['reg_date']).dt.days
    
    #  Убираем из датафрейма строки, где количество дней от момента регистрации за пределами исследуемой области
    if limit_days:
        full_users_data = full_users_data.query('DBRA <= @limit_days')
    
    #  Для каждого дня из выбранного диапазона определим количество зарегистрировавшихся пользователей
    #  Определим количество пользователей, которые зарегистрировались в одну дату 
    #  и зашли в игру через определённое количество дней                                          
    df_return_count = full_users_data.groupby(['reg_date','DBRA'], as_index=False).agg(users_return_count=('uid', 'nunique'))
        
    #  Изменим тип столбца reg_date                                           
    df_return_count['reg_date'] = df_return_count['reg_date'].astype(str)
    
    #  Создадим сводную таблицу
    df_retantion = df_return_count.pivot(index='reg_date', columns='DBRA', values='users_return_count')
    
    #  Рассчитаем Retantion 
    df_retantion = df_retantion.divide(df_retantion[0], axis=0).apply(lambda x: round(100 * x, 2))
    
    #  Найдём максимальное значение Retantion не считая 100%
    max_value = df_retantion.drop(0, axis=1).max().max()       
    
    # Строим тепловую карту
    plt.figure(figsize=(31, 10))
    sns.set(style='ticks', font_scale=1.2)
    ax = sns.heatmap(df_retantion, annot=True, vmin=0.0, vmax=max_value, fmt='', linewidths = 1.5, cmap="Oranges")
    if limit_days:
        plt.title(f"Тепловая карта retention (период {start_time} - {finish_time}, ограничение по количеству дней - {limit_days})", fontsize=15)
    else:
        plt.title(f"Тепловая карта retention (период {start_time} - {finish_time}", fontsize=15)
    plt.title(f"Тепловая карта retention (период {start_time} - {finish_time})", fontsize=15)
    ax.set_xlabel('Количество дней между регистрацией и авторизацией', fontsize=13)
    ax.set_ylabel('Дата регистрации', fontsize=13)
    plt.show()
    pass

NameError: name 'datetime' is not defined